# 1) Données

In [23]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 24
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        profils[i] = cardProfils
        cardProfils = cardProfils + 1
        
        
print(profils)
#print(cardProfils)
N = np.zeros(cardProfils)
for i in range(cardO):
    N[int(profils[i])] += 1
#print(N)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein

delta = np.full((cardProfils, cardJ), 1)
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    if d[index] != -1:
        d_i = d[index]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[profil][j_s] = 0
            
kappaProfils = []
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    kappaProfils.append(kappa[index])
    
creneaux = [0,2,1,2,0,1,3,1,0,2,1,2,0,1,3,1,0,1,2,1,0,3,1,2,0]
cardCreneaux = len(np.unique(creneaux))
alpha = []
for p in range(cardP):
    ligne = []
    for cren in range(cardCreneaux):
        if creneaux[p] == cren:
            ligne.append(1)
        else:
            ligne.append(0)
    alpha.append(ligne)

#print(nomsOperateurs)
#print(competencesOperateurs)
print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])
#print(kappaProfils)
#print(delta[0])
print(creneaux[21])

[0. 1. 2. 1. 1. 2. 3. 1. 2. 1. 4. 1. 1. 1. 1. 1. 2. 3. 1. 3. 2. 1. 1. 3.]
0           Auto 7h
1        Dech 12h45
2       Lav-cond-7h
3     Cond 2- 10h30
4             R-RTT
5        Dech 10h30
6        Cond 12h45
7           Dech 7h
8                 R
9     Lav2-Rc-10h30
10       Cond-2- 7h
11        Sté 10h30
12                R
13        Lav 1- 7h
14       Auto 12h45
15       Cond-1- 7h
16                R
17    Cond 1- 10h30
18        Lav 2- 7h
19                R
20             R-7h
21        Sté 12h45
22           Sté 7h
23     Lav 1- 10h30
Name: NOMPOSTE, dtype: object
3


# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for prof in range(0,cardProfils):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[prof,j,p]=LpVariable(f"x({prof,j,p})", cat=LpBinary) #LpBinary

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for prof in range(0,cardProfils):
    for r in range(0,cardR):    
        y[prof,r]=LpVariable(f"y({prof,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for prof in range(0,cardProfils):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[prof,s,p]=LpVariable(f"z({prof,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        u[p,s]=LpVariable(f"u({p,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  20160
nb de variables y_ir  =  120
nb de variables z_isp =  2880
nb de variables u_ir  =  576

nb total de variables =  23736


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModeleProfils", LpMinimize)

prob += sum(u[p,s] for p in range(0,cardP) for s in range(0,cardS)), "obj"

## 2.3) Contraintes

In [4]:
#contrainte 1 Un profil opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappaProfils[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un profil opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappaProfils[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, un profil opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardProfils):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                a = 0
                #prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,j,p}"

for j in range(cardJ):
    s = j // 7
    for i in range(cardProfils):
        for p in range(cardP):
            if rho[p] == 1 and o[j] == 1 and delta[i][j] == 1:
                #print("p=",p)
                a=0
                prob += z[i,s,p] <= x[i,j,p], f"C3bis_{j,i,p}"

#contrainte 4 Chaque jour, un profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) == N[i], f"C4_{i,j}"

#contrainte 5 Chaque jour, chaque poste non rouleur est assuré par exactement un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[i,j,p] for i in range(cardProfils)) == 1, f"C5_{j,p}"
            
#contrainte 6 Chaque jour, les postes rouleurs sont occupés par au plus un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardProfils)) <= 1, f"C6_{j,p}"

#contrainte 7 Chaque semaine, chaque profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) == N[i], f"C7_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un profil opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for i in range(0,cardProfils)) == 1, f"C8_{s,p}"

#contrainte 9 Chaque profil opérateur est affecté à autant de positions dans le roulement que son nombre de membres

for i in range(0,cardProfils):
    prob += sum(y[i,r] for r in range(0,cardR)) == N[i], f"C9_{i}"

#contrainte 10 Chaque position dans le roulement est associée à un profil opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardProfils)) == 1, f"C10_{r}"

#contrainte 11 Un profil opérateur peut permuter de poste si leurs horaires sont compatibles

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C11_{i,s,p}"

#contrainte 12 Lorsqu'un poste est occupé par un autre profil opérateur que celui prévu initialement,
#son score d'insatisfaction augmente
for s in range(0,cardS):
    for p in range(0,cardP):
        for i in range(cardProfils):
            prob += u[p,s] >= (kappaProfils[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C12_{i,s,p}"
            
#contrainte David anti permutation interdite
for s in range(cardS):
    for i in range(cardProfils):
        for c in range(cardCreneaux):
            prob += sum(alpha[p][c]*z[i,s,p] for p in range(cardP)) == sum(alpha[p][c]*y[i,rFunction(s,p)] for p in range(cardP)), f"C13_{s,i,c}"

for j in range(cardJ):
    s = j // 7
    for i in range(cardProfils):
        a = 0
        #prob += sum(x[i,j,p]*rho[p] for p in range(cardP)) <= sum(z[i,s,p]*rho[p] for p in range(cardP)), f"C14_{j,i,c}"


## 2.4) Résolution du problème

In [5]:
import time
import cplex
from cplex.callbacks import MIPInfoCallback
log = []

class MyCallback(MIPInfoCallback):

    def __call__(self):
        current = time.time()
        log.append([self.get_incumbent_objective_value(), current - start])

pathFile = "Modeles/ModeleExactProfils.lp"
prob.writeLP(pathFile)

c = cplex.Cplex()
c.parameters.timelimit.set(18000.0)
c.parameters.mip.cuts.cliques.set(3)
c.register_callback(MyCallback)
c.read(pathFile)

start = time.time()
c.solve()
#solution = c.solution
#print(solution.get_objective_value())
end = time.time()

duree = end - start
print()
print("Temps de résolution : " + str(duree))
print("Valeur objectif = ", c.solution.get_objective_value())

print()
print("log")
for i in range(len(log)):
    print(log[i])

newLog = []
currentSol = 10000
for i in range(len(log)):
    if log[i][0] < currentSol:
        currentSol = log[i][0]
        newLog.append(log[i])
newLog.append(log[len(log)-1])
 
print()
print("newLog")
for i in range(len(newLog)):
    print(newLog[i])
    
for i in range(len(newLog)):
    str1 = str(newLog[i][0]).replace('.', ',')
    print(str1)
print()
for i in range(len(newLog)):
    str1 = str(newLog[i][1]).replace('.', ',')
    print(str1)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Cuts_Cliques                        3
CPXPARAM_TimeLimit                               18000
Legacy callback                                  i
Tried aggregator 2 times.
MIP Presolve eliminated 27528 rows and 8064 columns.
Aggregator did 2160 substitutions.
Reduced MIP has 14141 rows, 13512 columns, and 61536 nonzeros.
Reduced MIP has 13512 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (86.53 ticks)
Found incumbent of value 432.000000 after 0.19 sec. (166.51 ticks)
Probing changed sense of 8304 constraints.
Probing time = 0.11 sec. (59.11 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 8568 rows and 6576 columns.
Reduced MIP has 5573 rows, 6936 columns, and 38640 nonzeros.
Reduced MIP has 6936 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (34.47 ticks)
Probing time = 0.01 sec. (5.34 ticks)
Tried aggregator 

*  1392+ 1190                           22.0000        7.1644            67.43%
   1401  1178        7.7393  2482       22.0000        7.1644  1438511   67.43%
   1444  1179        8.8201  2354       22.0000        7.2069  1473647   67.24%
   1500  1230        9.7293  1998       22.0000        7.2148  1543292   67.21%

Clique cuts applied:  0
Zero-half cuts applied:  37
Lift and project cuts applied:  7

Root node processing (before b&c):
  Real time             =   29.31 sec. (21587.50 ticks)
Parallel b&c, 8 threads:
  Real time             =   40.32 sec. (28712.00 ticks)
  Sync time (average)   =    5.25 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =   69.63 sec. (50299.50 ticks)

Temps de résolution : 69.63687086105347
Valeur objectif =  22.0

log
[115.0, 0.8070199489593506]
[115.0, 0.8072390556335449]
[41.0, 4.821357011795044]
[41.0, 4.8267011642456055]
[28.0, 6.578207969665527]
[28.0, 6.5848822593688965]
[28.0, 7.252960

# 3) Affichage de la solution

In [30]:
print("objectif =", c.solution.get_objective_value())
for s in range(cardS):
    for p in range(cardP):
        val = c.solution.get_values("u(("+str(p)+",_"+str(s)+"))")
        if val != 0:
            print("u[",p,",",s,"] =", val)

print()
for i in range(cardProfils):
    for r in range(cardR):
        val = c.solution.get_values("y(("+str(i)+",_"+str(r)+"))")
        if val != 0:
            print("y[",i,",",r,"] =", val)
        
print()
for s in range(cardS):
    for i in range(cardProfils):
        for p in range(cardP):
            val = c.solution.get_values("z(("+str(i)+",_"+str(s)+",_"+str(p)+"))")
            if val != 0:
                a = 0
                #print("z[",i,",",s,",",p,"] =", val)

#print(c.variables.get_names())
#solver.solution.get_objective_value()
#print(objval)

objectif = 22.0
u[ 19 , 0 ] = 1.0
u[ 6 , 4 ] = 1.0
u[ 6 , 5 ] = 1.0
u[ 19 , 5 ] = 1.0
u[ 6 , 6 ] = 1.0
u[ 7 , 6 ] = 1.0
u[ 2 , 7 ] = 1.0
u[ 7 , 7 ] = 1.0
u[ 5 , 8 ] = 1.0
u[ 2 , 9 ] = 1.0
u[ 6 , 11 ] = 1.0
u[ 6 , 12 ] = 1.0
u[ 13 , 12 ] = 1.0
u[ 5 , 13 ] = 1.0
u[ 18 , 13 ] = 1.0
u[ 9 , 14 ] = 1.0
u[ 3 , 15 ] = 1.0
u[ 6 , 16 ] = 1.0
u[ 10 , 17 ] = 1.0
u[ 3 , 18 ] = 1.0
u[ 13 , 19 ] = 1.0
u[ 6 , 23 ] = 1.0

y[ 0 , 19 ] = 1.0
y[ 1 , 1 ] = 1.0
y[ 1 , 2 ] = 1.0
y[ 1 , 4 ] = 1.0
y[ 1 , 6 ] = 1.0
y[ 1 , 7 ] = 1.0
y[ 1 , 13 ] = 1.0
y[ 1 , 14 ] = 1.0
y[ 1 , 15 ] = 1.0
y[ 1 , 16 ] = 1.0
y[ 1 , 18 ] = 1.0
y[ 1 , 20 ] = 1.0
y[ 1 , 21 ] = 1.0
y[ 1 , 23 ] = 1.0
y[ 2 , 0 ] = 1.0
y[ 2 , 3 ] = 1.0
y[ 2 , 5 ] = 1.0
y[ 2 , 11 ] = 1.0
y[ 2 , 12 ] = 1.0
y[ 3 , 8 ] = 1.0
y[ 3 , 9 ] = 1.0
y[ 3 , 10 ] = 1.0
y[ 3 , 22 ] = 1.0
y[ 4 , 17 ] = 1.0



# 4) Conversion de la solution vers des profils individuels (pour texte et CSV)

In [28]:
# Afin de retourner un planning exploitable par le CHU, nous avons besoin de convertir notre
# solution "par profils" en solution représentant chaque opérateur individuellement.

# on récupère pour chaque profil la composition de ses membres
groupesProfil = []
for profil in range(cardProfils):
    groupeProfil = []
    for i in range(cardO):
        if profils[i] == profil:
            groupeProfil.append(i)
    groupesProfil.append(groupeProfil)

print("groupesProfil = ", groupesProfil)

# on crée le nouveau vecteur Y, vecteurY[i] donne la position dans le roulement de l'opérateur i
vecteurY = np.zeros(cardO)

# pour chaque profil, on associe chacun de ses membres aux rangs du roulement qui lui sont associés
for prof in range(cardProfils):
    groupeProfil = groupesProfil[prof]
    roulements = []
    # on récupère la liste des rangs du roulement associés à ce profil
    for r in range(cardR):
        if c.solution.get_values("y(("+str(prof)+",_"+str(r)+"))") != 0:
            roulements.append(r)
    # on associe arbitrairement chaque membre du groupe à un des rangs du roulement
    for index in range(len(groupeProfil)):
        vecteurY[groupeProfil[index]] = roulements[index]

print("vecteurY =", vecteurY)


# on crée la nouvelle matrice Z, z[s][i] donne l'affectation de l'opérateur i la semaine s
newZ = []
ligne = []
for i in range(cardO):
    ligne.append(-1)
for s in range(cardS):
    newZ.append(ligne.copy())


# pour chaque profil, on va donner à chacun de ses membres son affectation hebdomadaire
for profil in range(cardProfils):
    
    groupeProfil = groupesProfil[profil].copy()
    for s in range(cardS):
        affectProfil = []
        #Pour chaque semaine, on regarde quels postes doivent prendre en charge le groupe
        for p in range(cardP):
            if c.solution.get_values("z(("+str(profil)+",_"+str(s)+",_"+str(p)+"))") != 0:
                affectProfil.append(p)

        # pour chaque opérateur du groupe, on lui donne si possible son affectation théorique selon le vecteur Y
        operateursLibres = groupeProfil.copy() #copie pour savoir qui n'a pas encore de poste
        for index in range(len(groupeProfil)):
            ope = groupeProfil[index]
            posteTheoriqueOpe = (vecteurY[ope] + s) % cardP #théoriquement, le rang du poste augmente de 1 chaque semaine
            if posteTheoriqueOpe in affectProfil:
                affectProfil.remove(posteTheoriqueOpe)
                operateursLibres.remove(ope)
                newZ[s][ope] = posteTheoriqueOpe
                
        # une fois les affectations théoriques terminées, on affecte les postes restants
        if len(affectProfil) >= 1:
            while len(affectProfil) > 0:
                # on étudie itérativement chacun des postes à pourvoir
                poste = affectProfil[0]
                creneauPoste = creneaux[poste]
                trouve = False
                index2 = 0
                # on cherche parmi les opérateurs libres quelqu'un situé sur le même créneau horaire
                while index2 < len(operateursLibres) and not trouve:
                    ope = operateursLibres[index2]
                    posteTheorique = int((vecteurY[ope] + s) % cardP)
                    if creneaux[posteTheorique] == creneauPoste:
                        trouve = True
                        # lorsqu'on en trouve un, on affecte cet opérateur au poste
                        newZ[s][ope] = poste
                        affectProfil.remove(poste)
                        operateursLibres.remove(ope)
                    else:
                        index2 += 1
                # si le modèle est correct, on trouvera toujours quelqu'un de libre sur le même créneau (ctr alpha)
                
                    
if False:
    print("newZ = ")
    for s in range(cardS):
        print(newZ[s])

# on crée la nouvelle matrice X, x[j][i] donne le poste occupé par l'opérateur i le jour j
newX = []
ligne = []
for i in range(cardO):
    ligne.append(-1)
for j in range(cardJ):
    newX.append(ligne.copy())

# pour chaque jour du planning et pour chaque profil, on associe les opérateurs à leurs postes
for j in range(cardJ):
    semaineJ = j // 7
    for profil in range(cardProfils):
        # on dresse la liste des postes que ce profil doit prendre en charge ce jour
        postesVacants = []
        for p in range(cardP):
            if c.solution.get_values("x(("+str(profil)+",_"+str(j)+",_"+str(p)+"))") != 0:
                postesVacants.append(p)
                
        # si le profil ne travaille pas ce jour, on ne l'affecte nulle part (RA)
        # sinon, on doit chercher qui doit être affecté à quel poste
        if delta[profil][j] == 1 and o[j] == 1:
            groupeProfil = groupesProfil[profil]
            operateursLibres = groupeProfil.copy()
            
            # pour chaque opérateur affecté à un poste non rouleur cette semaine, on l'y affecte ce jour
            for index in range(len(groupeProfil)):
                ope = groupeProfil[index]
                affectHebdo = newZ[semaineJ][ope]
                if rho[int(affectHebdo)] == 1:
                    newX[j][ope] = affectHebdo
                    operateursLibres.remove(ope)
                    postesVacants.remove(affectHebdo)
            
            # si il reste des opérateurs affectés à non postes rouleurs, on les affecte aux postes vacants
            for index in range(len(operateursLibres)):
                ope = operateursLibres[index]
                affectJour = postesVacants[index]
                newX[j][ope] = affectJour

if False:
    print("newX = ")
    for j in range(cardJ):
        print(newX[j])

print("Profils convertis en opérateurs individuels")

groupesProfil =  [[0], [1, 3, 4, 7, 9, 11, 12, 13, 14, 15, 18, 21, 22], [2, 5, 8, 16, 20], [6, 17, 19, 23], [10]]
vecteurY = [19.  1.  0.  2.  4.  3.  8.  6.  5.  7. 17. 13. 14. 15. 16. 18. 11.  9.
 20. 10. 12. 21. 23. 22.]
Profils convertis en opérateurs individuels


# 4) Extraction de la solution au format texte

In [16]:
#temp
for i in range(cardProfils):
    for j in range(cardJ):
        for p in range(cardP):
            if c.solution.get_values("x(("+str(i)+",_"+str(j)+",_"+str(p)+"))") != 0:
                a = 0
                #print("x(("+str(i)+",_"+str(j)+",_"+str(p)+"))")

print(newZ[3])
print(newX[21])
print(newX[22])
print(newX[23])
print(newX[24])
print(newX[25])
print(newX[26])
print(newX[27])

groupe = groupesProfil[1]
res = []
for index in range(len(groupe)):
    ope = groupe[index]
    res.append(newZ[0][ope])
print(res)


[22.0, 4.0, 3.0, 5.0, 7.0, 6.0, 11.0, 9.0, 8.0, 10.0, 20.0, 16.0, 17.0, 18.0, 19.0, 21.0, 14.0, 12.0, 23.0, 13.0, 15.0, 0.0, 2.0, 1.0]
[22.0, 4, 3.0, 5.0, 7.0, 6.0, 11.0, 9.0, 8, 10.0, 12, 16, 17.0, 18.0, 19.0, 21.0, 14.0, 0, 23.0, 13.0, 15.0, 20, 2.0, 1.0]
[-1, 8, 3.0, 5.0, 7.0, 6.0, 11.0, 9.0, 22, 10.0, 0, 16, 17.0, 18.0, 19.0, 21.0, 14.0, 12, 23.0, 13.0, 15.0, 20, 2.0, 1.0]
[22.0, 8, 3.0, 5.0, 7.0, 6.0, 11.0, 9.0, 4, 10.0, 12, 16, 17.0, 18.0, 19.0, 21.0, 14.0, 0, 23.0, 13.0, 15.0, 20, 2.0, 1.0]
[22.0, 4, 3.0, 5.0, 7.0, 6.0, 11.0, 9.0, 20, 10.0, 12, 8, 17.0, 18.0, 19.0, 21.0, 14.0, 0, 23.0, 13.0, 15.0, 16, 2.0, 1.0]
[22.0, 4, 3.0, 5.0, 7.0, 6.0, 11.0, 9.0, 16, 10.0, 12, 8, 17.0, 18.0, 19.0, 21.0, 14.0, 0, 23.0, 13.0, 15.0, 20, 2.0, 1.0]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[1.0, 2.0, 4.0, 6.0, 7.0, 13.0, 14.0, 15.0, 16.0, 18.0, 2

In [17]:
with open("solutionsTexte/solutionModeleExactProfils", "w") as file_object:
        file_object.write(str(cardO) + "\n")           #cardO
        file_object.write(str(cardP) + "\n")           #cardP
        file_object.write(str(cardR) + "\n")           #cardR
        file_object.write(str(cardS) + "\n")           #cardS
        ligne = ""
        for p in range(0,cardP):
            ligne = ligne + str(rho[p])
        file_object.write(ligne + "\n")                #rho
        
        ligne = ""
        for j in range(0,cardJ):
            ligne = ligne + str(int(o[j]))
        file_object.write(ligne + "\n")                #o_j
        
        for i in range(0,cardO):
            ligne = ""
            for p in range(0,cardP):
                ligne = ligne + str(kappa[i][p])
            file_object.write(ligne + "\n")            #kappa_ip
            
        for p in range(0,cardP):
            ligne = ""
            for p2 in range(0,cardP):
                ligne = ligne + str(sigma[p][p2])
            file_object.write(ligne + "\n")            #sigma_pp'
            
        for i in range(0,cardO):
            ligne = ""
            for j in range(0,cardJ):
                ligne = ligne + str(delta[int(profils[i])][j])
            file_object.write(ligne + "\n")            #delta_ij
        
        ##Avant
        ##file_object.write("x" + "\n")
        #for i in range(0,cardO):
        #    #file_object.write(str(i) + "\n")
        #    for j in range(0,cardJ):
        #        ligne = ""
        #        for p in range(0,cardP):
        #            ligne = ligne + str(int(c.solution.get_values("x(("+str(int(profils[i]))+",_"+str(j)+",_"+str(p)+"))")))
        #        file_object.write(ligne + "\n")        #x_ijp
        
        #file_object.write("y" + "\n")
        
        for i in range(0,cardO):
            for j in range(0,cardJ):
                ligne = ""
                poste = newX[j][i]
                if poste == -1:
                    for p in range(0,cardP):
                        ligne += "0"
                else:
                    for p in range(0,cardP):
                        if p == poste:
                            ligne += "1"
                        else:
                            ligne += "0"
                file_object.write(ligne + "\n")        #x_ijp
        
        for i in range(0,cardO):
            ligne = ""
            for r in range(0,cardR):
                if vecteurY[i] == r:
                    ligne += "1"
                else:
                    ligne += "0"
            file_object.write(ligne + "\n")            #y_ir
        
        #file_object.write("z" + "\n")
        for i in range(0,cardO):
            for s in range(0,cardS):
                ligne = ""
                for p in range(0,cardP):
                    if newZ[s][i] == p:
                        ligne += "1"
                    else:
                        ligne += "0"
                    #ligne = ligne + str(int(c.solution.get_values("z(("+str(int(profils[i]))+",_"+str(s)+",_"+str(p)+"))")))
                file_object.write(ligne + "\n")        #z_isp
        
        file_object.write((str(int(c.solution.get_objective_value()))) + "\n")
        
print("Solution texte extraite")

Solution texte extraite


# 5) Extraction de la solution au format CSV

In [11]:
import csv
with open('CSV/planningExactProfils.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if newX[j][i] == p:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)
print("Planning CSV exporté")

Planning CSV exporté


In [12]:
print("Fin de programme")

Fin de programme
